Gather all tickers

In [16]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from typing import List, Tuple

wiki_page: str = requests.get('https://en.wikipedia.org/wiki/List_of_American_exchange-traded_funds').text
soup: BeautifulSoup = BeautifulSoup(wiki_page,'lxml')

list_items = soup.select('li:contains("|")')
tickers: List[str] = []

for list_item in list_items:
    li_text: str = list_item.text
    start_index: int = li_text.find('|')
    end_index: int = li_text.find(')',start_index)
    tickers.append(li_text[start_index +1:end_index].strip())

tickers.append('^GSPTSE') # add S&P/TSX Composite as benchmark
print(tickers)

['ITOT', 'ACWI', 'IWV', 'SCHB', 'FNDB', 'VT', 'VTI', 'VXUS', 'VTHR', 'DIA', 'RSP', 'IOO', 'IVV', 'SPY', 'SHE', 'VOO', 'IWM', 'OEF', 'QQQ', 'CVY', 'RPG', 'RPV', 'IWB', 'IWF', 'IWD', 'IVV', 'IVW', 'IVE', 'PKW', 'PRF', 'SPLV', 'SCHX', 'SCHG', 'SCHV', 'SCHD', 'FNDX', 'SDY', 'VOO', 'VOOG', 'VOOV', 'VV', 'VUG', 'VTV', 'MGC', 'MGK', 'MGV', 'VONE', 'VONG', 'VONV', 'VIG', 'VYM', 'DTN', 'DLN', 'MDY', 'DVY', 'IWR', 'IWP', 'IWS', 'IJH', 'IJK', 'IJJ', 'PDP', 'SCHM', 'IVOO', 'IVOG', 'IVOV', 'VO', 'VOT', 'VOE', 'VXF', 'DON', 'IWC', 'IWM', 'IWO', 'IWN', 'IJR', 'IJT', 'IJS', 'SCHA', 'FNDA', 'VIOO', 'VIOG', 'VIOV', 'VB', 'VBK', 'VBR', 'VTWO', 'VTWG', 'VTWV', 'EEB', 'ECON', 'IDV', 'ACWX', 'BKF', 'EFA', 'EFG', 'EFV', 'SCZ', 'EEM', 'PID', 'SCHC', 'SCHE', 'SCHF', 'FNDF', 'FNDC', 'FNDE', 'DWX', 'VEA', 'VWO', 'VXUS', 'VEU', 'VSS', 'DEM', 'DGS', 'AAXJ', 'EZU', 'EPP', 'IEV', 'ILF', 'FEZ', 'VGK', 'VPL', 'HEDJ', 'DFE', 'AND', 'GXF', 'EWA', 'EWC', 'EWG', 'EIS', 'EWI', 'EWJ', 'EWD', 'EWL', 'EWP', 'EWU', 'DXJ', 'NOR

Fetch ticker info from Yahoo

In [3]:
import yfinance as yf

data: pd.DataFrame = yf.download(tickers=" ".join(tickers), period="5y", interval="1d", group_by='ticker')
print(data)

[*********************100%***********************]  425 of 425 completed

11 Failed downloads:
- RPX: No data found, symbol may be delisted
- FTGS: No data found, symbol may be delisted
- CRDT: No data found for this date range, symbol may be delisted
- BGU: No data found for this date range, symbol may be delisted
- RWG: No data found, symbol may be delisted
- IRV: No data found for this date range, symbol may be delisted
- WDTI: No data found, symbol may be delisted
- BABZ: No data found for this date range, symbol may be delisted
- YPRO: No data found, symbol may be delisted
- QEH: No data found, symbol may be delisted
- RRF: No data found, symbol may be delisted
                   IDU                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2014-11-17  112.580002  114.260002  112.580002  114.169998   97.533577   
2014-11-18  114.260002  115.0

Save output to file to prevent further network requests.

In [4]:
found_tickers: List[str] = data.columns.get_level_values(0).unique().to_list()

for found_ticker in found_tickers:
    data[found_ticker].to_csv(found_ticker + '.csv')


Read files back from directory.

In [27]:
import glob

csv_paths: List[str] = glob.glob('*.csv')
prices_df: pd.DataFrame = None

for csv_path in csv_paths:
    (ticker_id, extension) = csv_path.split(".", 1)
    df: pd.DataFrame = pd.read_csv(csv_path, index_col='Date', usecols=['Date', 'Adj Close'], header=0, parse_dates=True)
    df = df.rename(columns={'Adj Close': ticker_id})

    if prices_df is not None:
        prices_df = prices_df.join(df)
    else:
        prices_df = df

prices_df

,HYG,SCHZ,IBB,RPX,BRAF,SHE,AMLP,IWB,XLY,ITOT,...,BZF,ICSH,AUNZ,EPP,FNDF,FDN,VXUS,IYZ,GGBP,PDP
Date,,,,,,,,,,,,,,,,,,,,,
2014-11-17,70.170639,46.071697,95.479362,NaN,NaN,0.001636,12.679347,103.664345,64.425415,42.127644,...,16.863153,46.573074,18.734585,38.474758,23.699223,61.529999,43.457695,27.144938,NaN,40.051895
2014-11-18,70.032524,46.098259,97.598236,NaN,NaN,NaN,12.740665,104.246666,64.546425,42.344742,...,17.050852,46.573074,18.838930,38.370331,23.998434,61.599998,43.873810,27.153812,NaN,40.425022
2014-11-19,70.017143,46.009739,97.254471,NaN,NaN,0.000818,12.822423,104.073792,64.816437,42.254288,...,17.159519,NaN,18.544867,37.735786,23.936832,60.990002,43.787132,26.843136,NaN,40.326824
2014-11-20,70.024826,46.045151,97.191658,NaN,NaN,NaN,12.849676,104.310364,65.095734,42.385437,...,17.238451,NaN,18.497433,37.543015,23.787228,61.139999,43.639748,26.931902,NaN,40.523216
2014-11-21,70.408531,46.133648,97.518929,NaN,NaN,NaN,12.767918,104.838081,65.254013,42.584438,...,17.515156,NaN,18.621073,38.041016,24.060038,61.169998,44.168564,27.029543,NaN,40.552670
2014-11-24,70.400841,46.169071,99.218010,NaN,NaN,NaN,12.686161,105.202057,65.859154,42.733669,...,17.297823,46.461502,18.544989,37.832176,24.139240,61.549999,44.168564,27.011791,NaN,40.807968
2014-11-25,70.515999,46.231037,99.059341,NaN,NaN,NaN,12.652097,105.156563,66.063942,42.697502,...,17.445908,46.461502,18.573519,37.679558,24.192039,61.730000,44.263912,26.994032,NaN,40.847237
2014-11-26,70.608040,46.292988,100.447685,NaN,NaN,NaN,12.767918,105.365829,66.054642,42.819614,...,17.673218,NaN,18.544989,38.073147,24.253643,62.049999,44.489300,27.278088,NaN,40.974876
2014-11-28,69.932732,46.346107,100.457619,NaN,NaN,0.000818,12.331876,105.138359,66.855331,42.647758,...,17.110125,NaN,18.535475,37.390400,23.963232,62.150002,43.951828,27.313595,NaN,40.788326
